In [ ]:
import requests
import json


In [ ]:
requests.delete("http://localhost:9200/_ltr/")

In [ ]:
requests.put("http://localhost:9200/_ltr/")

In [ ]:
feature_set = {
   "featureset": {
      "features": [
         {
            "name": "title_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "title": "{{keywords}}"
                     }
               }
         },
         {
            "name": "overview_bm25",
            "params": [
               "keywords"
            ],
            "template": {
                     "match": {
                        "overview": "{{keywords}}"
                     }
               }
         }
      ]
   },
     "validation": {
      "index": "tmdb",
      "params": {
         "keywords": "rambo"
      }
   }
}

requests.put('http://localhost:9200/_ltr/_featureset/my_feature_set', 
             json=feature_set)

In [ ]:
search = {
    "query": { 
    "match": {
        "title": "First Blood"
    }}
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search).json()
print(resp['hits']['hits'][0]['_source']['title'])

In [ ]:
search_with_log = {
  "query": {
    "bool": {
      "filter": [
        {
          "sltr": {
            "_name": "logged_features",
            "featureset": "my_feature_set",
            "params": {
              "keywords": "rambo"
            }
          }
        },
         {
          "terms": {
            "_id": [
              "7555","1370", "1369", "1368"
            ]
          }
        }
      ]
    }
  },
  "ext": {
    "ltr_log": {
      "log_specs": {
        "name": "ltr_features",
        "named_query": "logged_features"
      }
    }
  }
}

resp = requests.get('http://localhost:9200/tmdb/_search', json=search_with_log).json()
print(json.dumps(resp['hits']['hits'][0], indent=2))

In [ ]:
model = """## LambdaMART
## No. of trees = 10
## No. of leaves = 10
## No. of threshold candidates = 256
## Learning rate = 0.1
## Stop early = 100

<ensemble>
	<tree id="1" weight="0.1">
		<split>
			<feature> 2 </feature>
			<threshold> 10.664251 </threshold>
			<split pos="left">
				<feature> 1 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> -1.8305741548538208 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 9.502127 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 7.0849166 </threshold>
						<split pos="left">
							<output> 0.23645669221878052 </output>
						</split>
						<split pos="right">
							<output> 1.7593677043914795 </output>
						</split>
					</split>
					<split pos="right">
						<output> 1.9719607830047607 </output>
					</split>
				</split>
			</split>
			<split pos="right">
				<feature> 2 </feature>
				<threshold> 0.0 </threshold>
				<split pos="left">
					<output> 1.3728954792022705 </output>
				</split>
				<split pos="right">
					<feature> 2 </feature>
					<threshold> 8.602512 </threshold>
					<split pos="left">
						<feature> 1 </feature>
						<threshold> 0.0 </threshold>
						<split pos="left">
							<feature> 2 </feature>
							<threshold> 13.815164 </threshold>
							<split pos="left">
								<output> 1.9401178359985352 </output>
							</split>
							<split pos="right">
								<output> 1.99532949924469 </output>
							</split>
						</split>
						<split pos="right">
							<feature> 1 </feature>
							<threshold> 11.085816 </threshold>
							<split pos="left">
								<output> 2.0 </output>
							</split>
							<split pos="right">
								<output> 1.99308180809021 </output>
							</split>
						</split>
					</split>
					<split pos="right">
						<output> 1.9870178699493408 </output>
					</split>
				</split>
			</split>
		</split>
	</tree>
</ensemble>
"""


create_model = {
  "model": {
     "name": "my_model",
     "model": {
         "type": "model/ranklib",
         "definition": model
    }
  }
}

requests.post('http://localhost:9200/_ltr/_featureset/my_feature_set/_createmodel',
              json=create_model).json()

In [ ]:
search = {
  "query": {
      "sltr": {
          "params": {
              "keywords": "rambo"
          },
          "model": "my_model"
      }
  }
}

In [ ]:
for hit in resp['hits']['hits']:
    print(hit['_source']['title'])
